### Preparing the Data

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

In [2]:
df= pd.read_csv('movie_data.csv', encoding= 'utf-8')

In [3]:
#step 1: create a dataset
target= df.pop('sentiment')
ds_raw= tf.data.Dataset.from_tensor_slices((df.values, target.values))

#inspection:
for ex in ds_raw.take(3):
    tf.print(ex[0].numpy()[0][:50], ex[1])

b'In 1974, the teenager Martha Moxley (Maggie Grace)' 1
b'OK... so... I really like Kris Kristofferson and h' 1
b'***SPOILER*** Do not read this, if you think about' 1


In [4]:
tf.random.set_seed(1)
ds_raw= ds_raw.shuffle(50000,reshuffle_each_iteration= False)
ds_raw_test= ds_raw.take(25000)
ds_raw_train_valid= ds_raw.skip(25000)
ds_raw_train= ds_raw_train_valid.take(20000)
ds_raw_valid= ds_raw_train_valid.skip(20000)

In [6]:
#step 2: find unique tokens (words)
from collections import Counter

tokenizer= tfds.features.text.Tokenizer()
token_counts= Counter()

for example in ds_raw_train:
    tokens= tokenizer.tokenize(example[0].numpy()[0])
    token_counts.update(tokens)

print('Vocab size:', len(token_counts))

Vocab size: 87007


In [9]:
#step 3: encoding unique tokens to integers
encoder= tfds.features.text.TokenTextEncoder(token_counts)
example_str= 'This is an example!'
print(encoder.encode(example_str))

#step 3-A: define the function for transformation
def encode(text_tensor, label):
    text= text_tensor.numpy()[0]
    encoded_text= encoder.encode(text)
    return encoded_text, label

#step 3-B: wrap the encode function to a TF operation
def encode_map_fun(text,label):
    return tf.py_function(encode, inp=[text, label], Tout= (tf.int64,tf.int64))
ds_train= ds_raw_train.map(encode_map_fun)
ds_valid= ds_raw_valid.map(encode_map_fun)
ds_test= ds_raw_test.map(encode_map_fun)

tf.random.set_seed(1)
for example in ds_train.shuffle(1000).take(5):
    print('Sequence length: ', example[0].shape)

[232, 9, 270, 1123]
Sequence length:  (24,)
Sequence length:  (179,)
Sequence length:  (262,)
Sequence length:  (535,)
Sequence length:  (130,)


In [10]:
#take a small subset
ds_subset= ds_train.take(8)
for example in ds_subset:
    print('Individual size: ', example[0].shape)

Individual size:  (119,)
Individual size:  (688,)
Individual size:  (308,)
Individual size:  (204,)
Individual size:  (326,)
Individual size:  (240,)
Individual size:  (127,)
Individual size:  (453,)


In [12]:
#dividing the dataset into batches
ds_batched= ds_subset.padded_batch(4, padded_shapes= ([-1],[]))

for batch in ds_batched:
    print('Batch Dimension: ', batch[0].shape)

Batch Dimension:  (4, 688)
Batch Dimension:  (4, 453)


In [13]:
train_data= ds_train.padded_batch(32, padded_shapes=([-1],[]))
valid_data= ds_valid.padded_batch(32, padded_shapes= ([-1],[]))
test_data= ds_test.padded_batch(32, padded_shapes= ([-1],[]))

In [15]:
from tensorflow.keras.layers import Embedding

model= tf.keras.Sequential()

model.add(Embedding(input_dim=100,output_dim=6,input_length=20, name='embed-layer'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, 20, 6)             600       
Total params: 600
Trainable params: 600
Non-trainable params: 0
_________________________________________________________________


### Building the RNN model

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dense

model= Sequential()
model.add(Embedding(input_dim=1000, output_dim=32))
model.add(SimpleRNN(32, return_sequences= True))
model.add(SimpleRNN(32))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          32000     
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 36,193
Trainable params: 36,193
Non-trainable params: 0
_________________________________________________________________


### Building an RNN model for the Sentimental Analysis task

In [18]:
embedding_dim=20
vocab_size= len(token_counts)+2
tf.random.set_seed(1)
#build the model
bi_lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,name='embed-layer'),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, name= 'lstm-layer'),name='bidir-lstm'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
bi_lstm_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, None, 20)          1740180   
_________________________________________________________________
bidir-lstm (Bidirectional)   (None, 128)               43520     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,792,021
Trainable params: 1,792,021
Non-trainable params: 0
_________________________________________________________________


In [20]:
#compile and train
bi_lstm_model.compile(optimizer= tf.keras.optimizers.Adam(1e-3),loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics= ['accuracy'])
history= bi_lstm_model.fit(train_data,validation_data= valid_data, epochs=10)

Epoch 1/10
625/625 [==============================] - 309s 495ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 2/10
625/625 [==============================] - 308s 492ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 3/10
625/625 [==============================] - 305s 488ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 4/10
625/625 [==============================] - 307s 492ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 5/10
625/625 [==============================] - 362s 579ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 6/10
625/625 [==============================] - 379s 606ms/step - loss: 1.1315e-09 - accuracy: 1.0000 - val_loss: 1.1317e-09 - val_accuracy: 1.0000
Epoch 7/10
625/625 [==============================] - 365s 584ms/step - loss

In [21]:
#evaluate results on the test data
test_results= bi_lstm_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(test_results[1]*100))

782/782 [==============================] - 62s 80ms/step - loss: 1.1318e-09 - accuracy: 1.0000
Test Acc.: 100.00%


In [22]:
from collections import Counter

def preprocess_datasets(ds_raw_train, ds_raw_valid, ds_raw_test, max_seq_length= None, batch_size=32):
    #step 1 is done
    
    #step 2 finding unique tokens
    tokenizer= tfds.features.text.Tokenizer()
    token_counts= Counter()
    for example in ds_raw_train:
        tokens= tokenizer.tokenize(example[0].numpy()[0])
        if max_seq_length is not None:
            tokens= tokens[-max_seq_length:]
        token_counts.update(tokens)
    print('Vocab size: ',len(token_counts))
    
    #step 3 encoding the texts
    encoder= tfds.features.text.TokenTextEncoder(token_counts)
    def encode(text_tensor, label):
        text= text_tensor.numpy()[0]
        encoded_text= encoder.encode(text)
        if max_seq_length is not None:
            encoded_text= encoded_text[-max_seq_length:]
        return encoded_text, label
    def encode_map_fn(text, label):
        return tf.py_function(encode, inp=[text,label], Tout= (tf.int64, tf.int64))
    ds_train= ds_raw_train.map(encode_map_fn)
    ds_valid= ds_raw_valid.map(encode_map_fn)
    ds_test= ds_raw_test.map(encode_map_fn)
    
    #step 4 batching the data sets
    train_data= ds_train.padded_batch(batch_size, padded_shapes= ([-1],[]))
    valid_data= ds_valid.padded_batch(batch_size, padded_shapes= ([-1],[]))
    test_data= ds_test.padded_batch(batch_size, padded_shapes= ([-1],[]))
    
    return (train_data, valid_data, test_data, len(token_counts))

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU

def build_rnn_model(embedding_dim, vocab_size, recurrent_type='SimpleRNN',n_recurrent_units=64, n_recurrent_layers=1,bidirectional= True):
    tf.random.set_seed(1)
    
    #building the model
    model= tf.keras.Sequential()
    model.add(Embedding(input_dim=vocab_size,output_dim= embedding_dim,name='embed-layer'))
    for i in range(n_recurrent_layers):
        return_sequences= (i<n_recurrent_layers-1)
        
        if recurrent_type=='SimpleRNN':
            recurrent_layer= SimpleRNN(units= n_recurrent_units, return_sequences=return_sequences, name='simprnn-layer-{}'.format(i))
        elif recurrent_type=='LSTM':
            recurrent_layer= LSTM(units= n_recurrent_units, return_sequences=return_sequences, name='lstm-layer-{}'.format(i))
        elif recurrent_type== 'GRU':
            recurrent_layer= GRU(units= n_recurrent_units, return_sequences=return_sequences, name='gru-layer-{}'.format(i))
        if bidirectional:
            recurrent_layer= Bidirectional(recurrent_layer, name='bidir-'+recurrent_layer.name)
        model.add(recurrent_layer)
    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

In [25]:
batch_size= 32
embedding_dim= 20
max_seq_length=100
train_data, valid_data, test_data, n= preprocess_datasets(ds_raw_train, ds_raw_valid, ds_raw_test, max_seq_length= max_seq_length, batch_size=batch_size)
vocab_size= n+2
rnn_model= build_rnn_model(embedding_dim, vocab_size, recurrent_type='SimpleRNN',n_recurrent_units=64,n_recurrent_layers=1, bidirectional= True)
rnn_model.summary()

Vocab size:  58063
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed-layer (Embedding)      (None, None, 20)          1161300   
_________________________________________________________________
bidir-simprnn-layer-0 (Bidir (None, 128)               10880     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 1,180,501
Trainable params: 1,180,501
Non-trainable params: 0
_________________________________________________________________


In [26]:
rnn_model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss= tf.keras.losses.BinaryCrossentropy(from_logits=False),metrics=['accuracy'])

In [27]:
history= rnn_model.fit(train_data,validation_data=valid_data,epochs=10)

Epoch 1/10
625/625 [==============================] - 35s 55ms/step - loss: 0.0045 - accuracy: 0.9995 - val_loss: 5.6361e-06 - val_accuracy: 1.0000
Epoch 2/10
625/625 [==============================] - 37s 59ms/step - loss: 3.5371e-06 - accuracy: 1.0000 - val_loss: 2.0999e-06 - val_accuracy: 1.0000
Epoch 3/10
625/625 [==============================] - 37s 59ms/step - loss: 1.3893e-06 - accuracy: 1.0000 - val_loss: 8.7793e-07 - val_accuracy: 1.0000
Epoch 4/10
625/625 [==============================] - 35s 56ms/step - loss: 6.0614e-07 - accuracy: 1.0000 - val_loss: 4.0597e-07 - val_accuracy: 1.0000
Epoch 5/10
625/625 [==============================] - 35s 56ms/step - loss: 2.9333e-07 - accuracy: 1.0000 - val_loss: 2.0733e-07 - val_accuracy: 1.0000
Epoch 6/10
625/625 [==============================] - 35s 57ms/step - loss: 1.5695e-07 - accuracy: 1.0000 - val_loss: 1.1786e-07 - val_accuracy: 1.0000
Epoch 7/10
625/625 [==============================] - 36s 58ms/step - loss: 9.0917e-08 - acc

In [28]:
results= rnn_model.evaluate(test_data)
print('Test Acc.: {:.2f}%'.format(results[1]*100))

782/782 [==============================] - 19s 24ms/step - loss: 2.7531e-08 - accuracy: 1.0000
Test Acc.: 100.00%
